# Preprocess data and add prior GRN information
In this notebook, we will go through the preprocessing steps needed prior to running RegVelo pipeline.

## Library import 

In [1]:
import scvelo as scv
import scanpy as sc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplscience
import seaborn as sns

import regvelo as rgv

/home/icb/yifan.chen/miniconda3/envs/regvelo-test/lib/python3.10/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


## Load data
Read murine neural crest data that contains `.layers['spliced']` and `.layers['unspliced']`.

In [2]:
adata = rgv.datasets.murine_nc(data_type = "velocyto")

  0%|          | 0.00/281M [00:00<?, ?B/s]

In [3]:
adata

AnnData object with n_obs × n_vars = 6788 × 30717
    obs: 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'UMI_count', 'gene_count', 'major_trajectory', 'celltype_update', 'UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_2d_1', 'UMAP_2d_2', 'terminal_state', 'nCount_intron', 'nFeature_intron'
    var: 'vf_vst_counts_mean', 'vf_vst_counts_variance', 'vf_vst_counts_variance.expected', 'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable', 'vf_vst_counts_rank', 'var.features', 'var.features.rank'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced'

## Data preprocessing
We perform preprocessing steps, consisting of filtering, normalization, and logarithmizing. We also compute the neighborhood graph in PCA space and embedd it for visualization with UMAP. We further compute the first and second order moments (means and uncentered variances) using `scv.pp.moments` needed for velocity estimation.

:::{note} If preprocessing steps have already performed, you can skip this section and proceed directly to loading prior GRN.

In [4]:
scv.pp.filter_genes(adata, min_shared_counts=20)
scv.pp.normalize_per_cell(adata)
scv.pp.filter_genes_dispersion(adata, n_top_genes=3000)
scv.pp.log1p(adata)

sc.pp.neighbors(adata,n_pcs = 30,n_neighbors = 50)
sc.tl.umap(adata)
scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

Filtered out 22217 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 3000 highly variable genes.


/tmp/ipykernel_694863/1257038519.py:4: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  scv.pp.log1p(adata)
2025-06-12 19:58:24.473074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749751105.469030  694863 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749751105.862566  694863 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749751107.048878  694863 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749751107.048948  694863 computation_pla

computing moments based on connectivities
    finished (0:00:02) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [5]:
adata

AnnData object with n_obs × n_vars = 6788 × 3000
    obs: 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'UMI_count', 'gene_count', 'major_trajectory', 'celltype_update', 'UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_2d_1', 'UMAP_2d_2', 'terminal_state', 'nCount_intron', 'nFeature_intron', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'vf_vst_counts_mean', 'vf_vst_counts_variance', 'vf_vst_counts_variance.expected', 'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable', 'vf_vst_counts_rank', 'var.features', 'var.features.rank', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'log1p', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced', 'Ms', 'Mu'
    obsp: 'distances', 'connectivities'

## Load prior GRN created from notebook 'Infer prior GRN from [pySCENIC](https://pyscenic.readthedocs.io/en/latest/installation.html)' for RegVelo
In the following, we load the processed prior GRN infromation into our AnnData object. In this step `.uns['skeleton']` and `.var['TF']` are added, which will be needed for RegVelo's velocity pipeline.

In [6]:
GRN = pd.read_parquet("regulon_mat_processed_all_regulons.parquet")

In [7]:
GRN.head()

0610005C13Rik  0610009L18Rik  0610010K14Rik  0610012G03Rik  \
0610005C13Rik              0              0              0              0   
0610009L18Rik              0              0              0              0   
0610010K14Rik              0              0              0              0   
0610012G03Rik              0              0              0              0   
0610030E20Rik              0              0              0              0   

               0610030E20Rik  0610038B21Rik  0610040B10Rik  0610040J01Rik  \
0610005C13Rik              0              0              0              0   
0610009L18Rik              0              0              0              0   
0610010K14Rik              0              0              0              0   
0610012G03Rik              0              0              0              0   
0610030E20Rik              0              0              0              0   

               0610043K17Rik  1110002L01Rik  ...  Zswim8  Zw10  Zwilch  Zwint  \
0610005C13Rik              0              0  ...       0     0       0      0   
0610009L18Rik              0              0  ...       0     0       0      0   
0610010K14Rik              0              0  ...       0     0       0      0   
0610012G03Rik              0              0  ...       0     0       0      0   
0610030E20Rik              0              0  ...       0     0       0      0   

               Zxdb  Zxdc  Zyg11b  Zyx  Zzef1  Zzz3  
0610005C13Rik     0     0       0    0      0     0  
0610009L18Rik     0     0       0    0      0     0  
0610010K14Rik     0     0       0    0      0     0  
0610012G03Rik     0     0       0    0      0     0  
0610030E20Rik     0     0       0    0      0     0  

[5 rows x 13697 columns]

:::{note} If preprocessing steps have already performed and you wish to retain the gene dimension, please set `keep_dim=True` in `rgv.pp.set_prior_grn`.

In [8]:
adata = rgv.pp.set_prior_grn(adata, GRN.T)

In [9]:
adata

AnnData object with n_obs × n_vars = 6788 × 2112
    obs: 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'UMI_count', 'gene_count', 'major_trajectory', 'celltype_update', 'UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_2d_1', 'UMAP_2d_2', 'terminal_state', 'nCount_intron', 'nFeature_intron', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'vf_vst_counts_mean', 'vf_vst_counts_variance', 'vf_vst_counts_variance.expected', 'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable', 'vf_vst_counts_rank', 'var.features', 'var.features.rank', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'log1p', 'neighbors', 'umap', 'regulators', 'targets', 'skeleton', 'network'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced', 'Ms', 'Mu'
    obsp: 'distances', 'connectivities'

In [10]:
velocity_genes = rgv.pp.preprocess_data(adata.copy()).var_names.tolist()

# select TFs that regulate at least one gene
TF = adata.var_names[adata.uns["skeleton"].sum(1) != 0]
var_mask = np.union1d(TF, velocity_genes)

adata = adata[:, var_mask].copy()

computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [11]:
adata = rgv.pp.filter_genes(adata)
adata = rgv.pp.preprocess_data(adata, filter_on_r2=False)

adata.var["velocity_genes"] = adata.var_names.isin(velocity_genes)
adata.var["TF"] = adata.var_names.isin(TF)

Number of genes: 1187
Number of genes: 1164


In [12]:
adata

AnnData object with n_obs × n_vars = 6788 × 1164
    obs: 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'UMI_count', 'gene_count', 'major_trajectory', 'celltype_update', 'UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_2d_1', 'UMAP_2d_2', 'terminal_state', 'nCount_intron', 'nFeature_intron', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'vf_vst_counts_mean', 'vf_vst_counts_variance', 'vf_vst_counts_variance.expected', 'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable', 'vf_vst_counts_rank', 'var.features', 'var.features.rank', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_genes', 'TF'
    uns: 'log1p', 'neighbors', 'umap', 'regulators', 'targets', 'skeleton', 'network'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced', 'Ms', 'Mu'
    obsp: 'distances', 'connectivities'

## Save data
This data, with the parameters chosen in this tutorial, can also be assessed by calling `rgv.datasets.murine_nc(data_type = "preprocessed")`

In [13]:
adata.write_h5ad("adata_processed_velo.h5ad")